# Weather Data Fetcher - Hungary Weather Data

## Overview

This notebook fetches historical weather data for Hungary from the Open-Meteo API. Weather conditions, especially temperature, have a strong correlation with electricity demand due to heating and cooling requirements.

### Data Details

- **Data Source**: Open-Meteo Historical Weather API
- **Country**: Hungary (multiple cities for weighted average)
- **Time Period**: 2015-2024
- **Temporal Resolution**: Hourly or daily (aggregated as needed)
- **Variables**: Temperature, relative humidity, and other meteorological parameters

### Weather Variables

1. **Temperature**: Air temperature (2 meters above ground) - primary driver of heating/cooling demand
2. **HDD (Heating Degree Days)**: Measures coldness - indicates heating energy demand
   - HDD = max(0, T_base - T_actual), where T_base is typically 18°C
3. **CDD (Cooling Degree Days)**: Measures hotness - indicates cooling energy demand
   - CDD = max(0, T_actual - T_base), where T_base is typically 22°C

### Importance for Load Forecasting

Weather is one of the most important external factors for electricity load forecasting because:
- **Heating**: During cold weather, electricity demand increases for heating systems
- **Cooling**: During hot weather, electricity demand increases for air conditioning
- **Comfort**: People adjust their energy consumption based on weather conditions

The relationship between temperature and electricity demand typically shows a U-shaped curve, with increased demand at both very cold and very hot temperatures.


In [1]:
import sys
sys.path.append('../src')

from weather_data_fetcher import WeatherDataFetcher
from datetime import datetime
import pandas as pd

In [2]:
fetcher = WeatherDataFetcher()

# City bilgilerini göster
print("Cityler ve Nüfusları:")
for city, info in fetcher.CITIES.items():
    print(f"  {city}: {info['population']:,} population")

Şehirler ve Nüfusları:
  Budapest: 1,750,000 nüfus
  Debrecen: 200,000 nüfus
  Szeged: 160,000 nüfus
  Miskolc: 150,000 nüfus
  Pecs: 140,000 nüfus


In [3]:
start_date = datetime(2015, 1, 1)
end_date = datetime(2024, 12, 31)

print(f"Veri fetchilecek tarih aralığı: {start_date.date()} - {end_date.date()}")

Veri çekilecek tarih aralığı: 2015-01-01 - 2024-12-31


In [4]:
# Sadece temperature datasini fetch ve işle
temp_df, city_data = fetcher.fetch_and_process_temperature(start_date, end_date, show_progress=True)


🌡️ 5 şehir için sıcaklık verisi çekiliyor...

[1/5] Budapest... 

2025-12-10 19:54:29,335 - INFO - ✓ Budapest: 87672 saatlik kayıt


✓ 87672 kayıt
[2/5] Debrecen... 

2025-12-10 19:54:57,104 - INFO - ✓ Debrecen: 87672 saatlik kayıt


✓ 87672 kayıt
[3/5] Szeged... 

...

✓ 87672 kayıt
[5/5] Pecs... 

2025-12-10 19:55:53,720 - INFO - ✓ Pecs: 87672 saatlik kayıt


✓ 87672 kayıt


📊 Nüfusa göre ağırlıklı ortalama sıcaklık hesaplanıyor...


2025-12-10 19:55:56,546 - INFO - Ağırlıklı ortalama hesaplandı: 87672 kayıt


🔄 Saatlik veri 15 dakikalığa indiriliyor...

✅ Sıcaklık verisi işlendi!
   Toplam kayıt sayısı: 350,688



In [5]:
print("\n📊 Sıcaklık Verisi:")
print(temp_df.head(10))
print(f"\nToplam rows sayısı: {len(temp_df):,}")
print(f"Date range: {temp_df['datetime'].min()} - {temp_df['datetime'].max()}")
print(f"Sıcaklık aralığı: {temp_df['temperature_2m'].min():.2f}°C - {temp_df['temperature_2m'].max():.2f}°C")


📊 Sıcaklık Verisi:
                   datetime  temperature_2m
0 2015-01-01 00:00:00+00:00       -7.208750
1 2015-01-01 00:15:00+00:00       -7.241354
2 2015-01-01 00:30:00+00:00       -7.273958
3 2015-01-01 00:45:00+00:00       -7.306563
4 2015-01-01 01:00:00+00:00       -7.339167
5 2015-01-01 01:15:00+00:00       -7.406667
6 2015-01-01 01:30:00+00:00       -7.474167
7 2015-01-01 01:45:00+00:00       -7.541667
8 2015-01-01 02:00:00+00:00       -7.609167
9 2015-01-01 02:15:00+00:00       -7.646250

Toplam kayıt sayısı: 350,688
Tarih aralığı: 2015-01-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00
Sıcaklık aralığı: -15.21°C - 37.50°C


In [6]:
# Sıcaklık datasini save
fetcher.save_csv(temp_df, "../data/raw/hungary_temperature_2015_2024.csv")

print("\n✅ Sıcaklık datasi saved!")

2025-12-10 19:55:57,704 - INFO - ✅ Veri kaydedildi: ../data/raw/hungary_temperature_2015_2024.csv



✅ Sıcaklık verisi kaydedildi!


In [7]:
# Sadece HDD/CDD datasini fetch ve işle
hdd_cdd_df = fetcher.fetch_and_process_hdd_cdd(start_date, end_date, city_data=city_data, show_progress=True)

2025-12-10 19:55:57,918 - INFO - calculate_daily_hdd_cdd: 3653 günlük kayıt hesaplandı



🔄 Şehir verileri 15 dakikalığa indiriliyor...
🌡️ Günlük HDD/CDD değerleri hesaplanıyor...


2025-12-10 19:55:57,920 - INFO - Budapest: 3653 günlük HDD/CDD hesaplandı
2025-12-10 19:55:57,937 - INFO - calculate_daily_hdd_cdd: 3653 günlük kayıt hesaplandı
2025-12-10 19:55:57,938 - INFO - Debrecen: 3653 günlük HDD/CDD hesaplandı
2025-12-10 19:55:57,957 - INFO - calculate_daily_hdd_cdd: 3653 günlük kayıt hesaplandı
2025-12-10 19:55:57,957 - INFO - Szeged: 3653 günlük HDD/CDD hesaplandı
2025-12-10 19:55:57,973 - INFO - calculate_daily_hdd_cdd: 3653 günlük kayıt hesaplandı
2025-12-10 19:55:57,974 - INFO - Miskolc: 3653 günlük HDD/CDD hesaplandı
2025-12-10 19:55:57,991 - INFO - calculate_daily_hdd_cdd: 3653 günlük kayıt hesaplandı
2025-12-10 19:55:57,992 - INFO - Pecs: 3653 günlük HDD/CDD hesaplandı
2025-12-10 19:55:58,116 - INFO - calculate_weighted_hdd_cdd: 3653 günlük ağırlıklı HDD/CDD hesaplandı


📅 Günlük HDD/CDD değerleri 15 dakikalığa genişletiliyor...

✅ HDD/CDD verisi işlendi!
   Toplam kayıt sayısı: 350,688



In [8]:
print("\n🌡️ HDD/CDD Verisi:")
print(hdd_cdd_df.head(10))
print(f"\nToplam rows sayısı: {len(hdd_cdd_df):,}")
print(f"Date range: {hdd_cdd_df['datetime'].min()} - {hdd_cdd_df['datetime'].max()}")
print(f"\nHDD istatistikleri:")
print(f"  Min: {hdd_cdd_df['hdd'].min():.2f}")
print(f"  Max: {hdd_cdd_df['hdd'].max():.2f}")
print(f"  Ortalama: {hdd_cdd_df['hdd'].mean():.2f}")
print(f"\nCDD istatistikleri:")
print(f"  Min: {hdd_cdd_df['cdd'].min():.2f}")
print(f"  Max: {hdd_cdd_df['cdd'].max():.2f}")
print(f"  Ortalama: {hdd_cdd_df['cdd'].mean():.2f}")


🌡️ HDD/CDD Verisi:
                   datetime        hdd  cdd
0 2015-01-01 00:00:00+00:00  22.056734  0.0
1 2015-01-01 00:15:00+00:00  22.056734  0.0
2 2015-01-01 00:30:00+00:00  22.056734  0.0
3 2015-01-01 00:45:00+00:00  22.056734  0.0
4 2015-01-01 01:00:00+00:00  22.056734  0.0
5 2015-01-01 01:15:00+00:00  22.056734  0.0
6 2015-01-01 01:30:00+00:00  22.056734  0.0
7 2015-01-01 01:45:00+00:00  22.056734  0.0
8 2015-01-01 02:00:00+00:00  22.056734  0.0
9 2015-01-01 02:15:00+00:00  22.056734  0.0

Toplam kayıt sayısı: 350,688
Tarih aralığı: 2015-01-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00

HDD istatistikleri:
  Min: 0.00
  Max: 29.87
  Ortalama: 7.00

CDD istatistikleri:
  Min: 0.00
  Max: 8.57
  Ortalama: 0.47


In [9]:
# HDD/CDD datasini save
fetcher.save_csv(hdd_cdd_df, "../data/raw/hungary_hdd_cdd_2015_2024.csv")

print("\n✅ HDD/CDD datasi saved!")

2025-12-10 19:56:00,280 - INFO - ✅ Veri kaydedildi: ../data/raw/hungary_hdd_cdd_2015_2024.csv



✅ HDD/CDD verisi kaydedildi!


In [10]:
# Temperature ve HDD/CDD datalerini datetime üzerinden merge
weather_df = pd.merge(temp_df, hdd_cdd_df, on='datetime', how='inner')

print('\n📊 Birleştirilmiş Weather Verisi:')
print(weather_df.head(10))
print(f'\nToplam rows sayısı: {len(weather_df):,}')
print(f'Date range: {weather_df["datetime"].min()} - {weather_df["datetime"].max()}')
print(f'\nSütunlar: {list(weather_df.columns)}')


📊 Birleştirilmiş Weather Verisi:
                   datetime  temperature_2m        hdd  cdd
0 2015-01-01 00:00:00+00:00       -7.208750  22.056734  0.0
1 2015-01-01 00:15:00+00:00       -7.241354  22.056734  0.0
2 2015-01-01 00:30:00+00:00       -7.273958  22.056734  0.0
3 2015-01-01 00:45:00+00:00       -7.306563  22.056734  0.0
4 2015-01-01 01:00:00+00:00       -7.339167  22.056734  0.0
5 2015-01-01 01:15:00+00:00       -7.406667  22.056734  0.0
6 2015-01-01 01:30:00+00:00       -7.474167  22.056734  0.0
7 2015-01-01 01:45:00+00:00       -7.541667  22.056734  0.0
8 2015-01-01 02:00:00+00:00       -7.609167  22.056734  0.0
9 2015-01-01 02:15:00+00:00       -7.646250  22.056734  0.0

Toplam kayıt sayısı: 350,688
Tarih aralığı: 2015-01-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00

Sütunlar: ['datetime', 'temperature_2m', 'hdd', 'cdd']


In [11]:
# Birleştirilmiş datayi save
fetcher.save_csv(weather_df, '../data/raw/hungary_weather_2015_2024.csv')

print('\n✅ Birleştirilmiş weather datasi saved!')

2025-12-10 19:56:41,423 - INFO - ✅ Veri kaydedildi: ../data/raw/hungary_weather_2015_2024.csv



✅ Birleştirilmiş weather verisi kaydedildi!
